# Detección de fraude
---
Esta parte de la evaluación consiste en realizar un modelo de predicción de fraude.

El set de datos està en `data/fraud/creditcard_train.csv`

La evaluación es tipo *datathon* de forma que las notas se calcularán en base a:

1. Al ranking de métricas de los modelos (80%)
2. Legibilidad y presentación del código (20%)

La forma de entrega será generar un fichero csv en formato a partir de las predicciones realizadas sobre el fichero `data/fraud/creditcard_test.csv`.

| IdObservación | clase_predicha | probabilidad_clase_1 |
| ------------- | ------------- | ------------- |
|00001|True|0.6398|
|00002|True|0.5892|
|00003|False|0.2163|

EL fichero resultante tiene que tener por nombre `inicialapellido_test.csv` y se tiene que enviar por email a:

datathonuib@gmail.com

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Análisis descriptivo básico

### Preparación de los datos

In [ ]:
df_train = pd.read_csv('data/fraud/creditcard_train.csv')

In [ ]:
df_train.head(5)

Eliminamos la columna **Index** pues no ofrece información, además ya la estructura de data frame de **Pandas** ya proporciona dicha información.

In [ ]:
df_train = df_train.drop('index', axis = 1)

### Estadísticos básicos del conjunto de datos

In [ ]:
df_train.describe()

En la tabla anterior, tenemos información como la media, la desviación típica, el valor mínimo, máximo y los cuartiles para tener una primera aproximación sobre los datos que se van a manejar.

### Mapa de calor de correlaciones

In [ ]:
# Getting and showing correlation matrix
corr = df_train.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=0.3, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5})

Como podemos observar del mapa de calor anterior, no existen correlaciones demasiado elevadas entre las variables. De hecho, hay una gran parte de variables que tienen una correlación practicamente nula. 
Debido a ello, no tendría sentido aplicar **Análisis de componentes principales (PCA)** para reducir la dimensionalidad.

In [ ]:
class_distribution = df_train['Class'].value_counts()
class_distribution / sum(class_distribution)

Nos encontramos ante un problema desbalanceado, ya que solo un ~5% de los datos pertenecen a la clase que queremos predecir.

# Modelización de los datos

En este punto, utilizaremos una serie de modelos estadísticos donde a la postre los compararemos mediante una serie de métricas tales como **AUC** o **ROC** que nos permitirán escoger el mejor modelo.

Debido a que disponemos un conjunto de datos de prueba que no contiene el valor real de la variable respuesta, utilizaremos el conjunto de entrenamiento para realizar las pruebas en primera instancia y obtener una aproximación del error.

Una vez elegido el mejor modelo, utilizaremos el conjunto de prueba real sobre el que deseamos realizar la predicción.

Los modelos elegidos son:

* Regresión logística
* Random Forest
* Boosting

Al trabajar con datos desbalanceados se tendrán que aplicar algunas correcciones para los métodos de regresión logística y de random forest, pero esto no será necesario para Boosting ya que por su propia naturaleza funciona bien en estos casos.

In [ ]:
from sklearn.model_selection import train_test_split
rnd_seed = 123

df = df_train
X_train, X_test, y_train, y_test = train_test_split(df.drop('Class', axis=1), 
                                                    df.Class, 
                                                    test_size=0.1, 
                                                    random_state=rnd_seed)
df_test = pd.read_csv('data/fraud/creditcard_test.csv')
df_test = df_test.drop('index', axis = 1)

Como calcular las estadísticas de cada clasificador se hace del mismo modo, se ha escrito una función. Esta función recibe dos parámetros:
- `y_true`: Target que queremos predecir.
- `y_proba`: Para cada observación, probabilidad que asigna nuestro predictor al target que queremos predecir.

A partir de estos dos parámetros la función va a dibujar la curva AUC-ROC y la relación entre precisión y _recall_.

In [ ]:
def plotStats(y_true, y_proba):
    fpr, tpr, thre = roc_curve(y_true, y_proba[:, 1])
    prec, rec, thre = precision_recall_curve(y_true=y_true, probas_pred=y_proba[:,1])
    roc_auc = roc_auc_score(y_true, y_proba[:, 1])

    plt.subplot(1, 2, 1)
    plt.plot(thre, prec[:-1], label='precision')
    plt.plot(thre, rec[:-1], label='recall')
    plt.legend(loc='best')
    plt.xlabel('Threshold')

    plt.subplot(1, 2, 2)
    plt.plot(fpr, tpr, label='AUC = {a}'.format(a=round(roc_auc, 3)))
    plt.plot([0, 1], '--', c='black', label='Random Classifier')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='best')

Del mismo modo, también se puede automatizar el envío de los resultados para evaluar los modelos.

In [ ]:
def preparar_output(modelo, df_test, file_prefix):
    clases = modelo.predict(df_test)
    probas = modelo.predict_proba(df_test)
    results = {
        "idObservación": df_test.index,
        "clase_predicha": clases,
        "probabilidad_clase_1": probas[:, 1]
    }
    df = pd.DataFrame(data = results)
    df[["idObservación", "clase_predicha", "probabilidad_clase_1"]].to_csv('data/fraud/' + file_prefix + '_test.csv')

### Regresión Logística

In [ ]:
from sklearn.linear_model    import LogisticRegression
from sklearn.metrics         import *

In [ ]:
logreg = LogisticRegression(random_state=rnd_seed, class_weight='balanced')
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

Es importante especificar el argumento `class_weight='balanced'` para que la librería se encargue de ajustar el peso de las clases de forma inversamente proporcional a `n_samples / (n_classes * np.bincount(y))`.

Como cosecuencia, nuestro modelo tendrá un _accuracy_ menor, pero va a mejorar su predicción fuera de la muestra.

In [ ]:
print(classification_report(y_test, y_pred))

Usando el conjunto de test que hemos creado (`X_test` y `y_test`) vamos a simular la precisión que tendría nuestro modelo fuera de la muestra:

In [ ]:
y_proba = logreg.predict_proba(X_test)
plotStats(y_test, y_proba)

In [ ]:
preparar_output(logreg, df_test, 'acrespijherreroLR')

### Random forest

In [ ]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble        import RandomForestClassifier
from sklearn.tree            import DecisionTreeClassifier

In [ ]:
# params = {'max_depth': range(1, 10),
#           'min_samples_split': range(2, 20, 2),
#           'min_samples_leaf': range(2, 40, 2)}

# classifier = RandomForestClassifier(random_state=rnd_seed, class_weight='balanced')
# randomForest = GridSearchCV(estimator=classifier, param_grid=params, cv=5, scoring='roc_auc')
# randomForest.fit(X_train, y_train)
# print(randomForest.best_params_)
# Output: {'max_depth': 9, 'min_samples_leaf': 2, 'min_samples_split': 4}

randomForest = RandomForestClassifier(random_state=rnd_seed, class_weight='balanced', 
                                      max_depth = 9, min_samples_leaf = 2, min_samples_split = 2)
randomForest.fit(X_train, y_train)

In [ ]:
y_pred = randomForest.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
y_proba = randomForest.predict_proba(X_test)
plotStats(y_test, y_proba)

In [ ]:
preparar_output(randomForest, df_test, 'acrespijherreroRF')

### Decision Tree

In [ ]:
# params = {'max_depth': range(1, 10),
#           'min_samples_split': range(2, 20, 2),
#           'min_samples_leaf': range(2, 40, 2)
#          }

# classifier = DecisionTreeClassifier()
# decisionTree = GridSearchCV(estimator=classifier, param_grid=params, cv=5, scoring='roc_auc')
# decisionTree.fit(X_train, y_train)
# print(decisionTree.best_params_)
# Output: {'max_depth': 2, 'min_samples_leaf': 16, 'min_samples_split': 2}

decisionTree = DecisionTreeClassifier(random_state=rnd_seed, class_weight='balanced',
                                      max_depth = 2, min_samples_leaf = 16, min_samples_split = 2)
decisionTree.fit(X_train, y_train)

In [ ]:
y_pred = decisionTree.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
y_proba = decisionTree.predict_proba(X_test)
plotStats(y_test, y_proba)

In [ ]:
preparar_output(decisionTree, df_test, 'acrespijherreroDT')

### Boosting + Log. Reg.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
logreg = LogisticRegression(random_state=rnd_seed, class_weight='balanced')
boostedLogReg = AdaBoostClassifier(logreg, random_state = rnd_seed)
boostedLogReg.fit(X_train, y_train)

In [ ]:
y_pred = boostedLogReg.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
y_proba = boostedLogReg.predict_proba(X_test)
plotStats(y_test, y_proba)

In [ ]:
preparar_output(boostedLogReg, df_test, 'acrespijherreroBLR')